# The Caldeira Leggett model

The Caldeira-Leggett model describes a quantum system
which experiences a linear coupling to an environment.

In this example, we consider a quantum system in a harmonic
potential which is experiencing caldeira leggett type interactions.

First we define the particular system we want to simulate.
We start the system in the groundstate of the Hamiltonian.


In [ ]:
import numpy as np
from scipy.constants import Boltzmann, hbar  # type: ignore stubs
from slate_core import FundamentalBasis
from slate_core.metadata import (
    Domain,
    spaced_volume_metadata_from_stacked_delta_x,
)

from slate_quantum import operator
from slate_quantum.dynamics.caldeira_leggett import CaldeiraLeggettCondition
from slate_quantum.metadata import SpacedTimeMetadata

# A basis with 60 points in the x direction
metadata = spaced_volume_metadata_from_stacked_delta_x((np.array([2 * np.pi]),), (61,))

potential = operator.build.harmonic_potential(metadata, 5, offset=(np.pi,))
mass = hbar**2
hamiltonian = operator.build.kinetic_hamiltonian(potential, mass)
eigenstates = operator.get_eigenstates_hermitian(hamiltonian)


condition = CaldeiraLeggettCondition(
    mass=mass,
    friction=0.001 / hbar,
    temperature=8 / Boltzmann,
    potential=potential,
    initial_state=eigenstates[0, :],
)
times = FundamentalBasis(SpacedTimeMetadata(60, domain=Domain(delta=5 * np.pi * hbar)))

We then simulate a single realization using the stochastic schrodinger equation


In [ ]:
from IPython.display import HTML
from slate_core.plot import animate_data_over_list_1d_x

from slate_quantum.dynamics import (
    select_realization,
    simulate_caldeira_leggett_realizations,
)

realizations = simulate_caldeira_leggett_realizations(condition, times)
evolution = select_realization(realizations, 0)

# The state changes in time, but only by a phase difference if there is no friction
fig, ax, _anim0 = animate_data_over_list_1d_x(evolution, measure="real")
ax.set_title("Real part of the state")
HTML(_anim0.to_jshtml())

# If we plot the absolute value of the state, we see no change with time if there is no friction
fig, ax, _anim1 = animate_data_over_list_1d_x(evolution, measure="abs")
ax.set_title("Abs state, which would not change with time if there is no friction")
HTML(_anim1.to_jshtml())
